数据来源于 WikiText-2，一个较小的数据集，特点是：
1. 保留了原来的标点符号 ， 适合于下一句预测 。
2. 保留了原来的大小写和数字 。
3. 每行代表一个段落 ，任意标点符号及其前后的词元之间都有空格 。
4. 每个段落最少有两句话 。

在定制的数据集上对BERT进行预训练变得越来越流行，（启发：能否换一个数据集发论文）

In [1]:
import os
import random
import torch
from d2l import torch as d2l

简单起见，仅用句号`.`作为分隔符来拆分句子。还有更复杂的拆分句子的技术。

In [2]:
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')


#@save
def _read_wiki(data_dir):
    # 根据目录名和文件名构建路径
    file_name = os.path.join(data_dir, 'wiki.train.tokens')

    # 以只读模式打开
    # 并使用 `as` 关键字将打开的文件对象赋值（as 实现向右赋值）给变量 `f`
    # `with` 语句用于确保文件在使用完毕后被正确关闭。
    with open(file_name, 'r') as f:
        # 读取文件的所有行，并将其存储在变量 `lines` 中。每一行作为一个字符串元素存储在列表中。
        lines = f.readlines()
    # `line.strip()：去除每一行两端的空格和换行符。
    # .lower()：将每一行中的大写字母转换为小写字母。
    # .split(' . ')：使用字符串 `' . '` 作为分隔符将每一行拆分成一个段落中的句子列表。
    # for line in lines if len(line.split(' . ')) >= 2`：对于长度大于等于2的行（含有至少两个句子），进行处理。
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

In [3]:
s = ["a","b","c","d","e"]
random.shuffle(s)
s

['b', 'c', 'a', 'e', 'd']

## 1.定义辅助函数
首先为BERT的两个预训练任务mlm、nsp 实现辅助函数。
这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

### (1) 生成下一句预测任务的数据
BERT 的 NSP 类中有提到：
> 在为预训练生成句子对时，有一半的时间它们确实是标签为“真”的连续句子；
> 在另一半的时间里，第二个句子是从语料库中随机抽取的，标记为“假”。

In [4]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套(段落数,每段句子数,每句单词数),选中元素 paragraphs[random_i][random_j]
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

下面的函数通过调用 `_get_next_sentence` 函数从输入 paragraph 生成用于下一句预测的训练样本。
这里 `paragraph` 是句子列表，其中每个句子都是词元列表。
自变量 `max_len` 指定预训练期间的 BERT 输入序列的最大长度。

In [5]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

### (2) 生成 MLM 任务的数据

In [6]:
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break

        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        # 完成遮蔽操作
        mlm_input_tokens[mlm_pred_position] = masked_token

        # 记录下 candidate_pred_positions 单词的下标及原值
        pred_positions_and_labels.append((mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

通过调用前述的_replace_mlm_tokens函数，以下函数将BERT输入序列（tokens）作为输入，并返回
1. 输入词元的索引（在可能的词元替换之后）
2. 发生预测的词元索引
3. 以及这些预测的标签索引。

In [7]:
def _get_mlm_data_from_tokens(tokens, vocab):
    # 记录(除了特殊词元外)待预测单词的下标
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)

    # MLM 任务中预测一个段落中 15% 的随机词元，超出次数不再预测
    num_mlm_preds = max(1, round(len(tokens) * 0.15))  # round 是四舍五入
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)

    # 按照预测词元的下标排序
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]

    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

## 二、将文本转换为预训练数据集
辅助函数 `_pad_bert_inputs()` 来将特殊的 `<mask>` 词元附加到输入。
其参数`examples`包含来自两个预训练任务的辅助函数 `_get_nsp_data_from_paragraph` 和 `_get_mlm_data_from_tokens` 的输出。
主要工作：将长度不一的句子统统填充长度到`<pad>`  max_len 。其它相关变量也同样处理

In [8]:
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens, = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []

    for (token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        # 句子有长有短，为了保持数据长度一致，在不足 max_len 长的句子后面补 '<pad>'
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
                max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
                max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数，valid_lens 表示每句中自然词元的个数。
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))

        # 汇集所有段落的预测位置，亦做填充至 max_num_mlm_preds 等长
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
                max_num_mlm_preds - len(pred_positions)), dtype=torch.long))

        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                    max_num_mlm_preds - len(pred_positions)),
                         dtype=torch.float32))

        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
                max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))

        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))

    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

将用于`生成两个预训练任务的训练样本` 的辅助函数和用于 `填充输入` 的辅助函数放在一起，
定义以下_WikiTextDataset类为用于预训练BERT的WikiText-2数据集。
通过实现`__getitem__`函数，我们可以任意访问 WikiText-2 语料库的一对句子生成的预训练样本（遮蔽语言模型和下一句预测）样本。

最初的BERT模型使用词表大小为 `30000` 的 WordPiece 嵌入。
为简单起见，我们使用 d2l.tokenize 函数进行词元化。出现次数少于5次的不频繁词元将被过滤掉。

In [9]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出 paragraphs[i] 是代表段落的句子列表，其中每个句子都是词元列表。paragraphs[i][j][k] 表示第 i 个段落中第 j 句的第 k 个词元
        # sentences 是一个二维张量，把所有段落中的句子整合到一起。sentences[i][j] 表示第 i 个句子中第 j 个单词
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])

        # 获取 NSP 任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))

        # 获取 MLM 任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                     + (segments, is_next))
                    for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = \
            _pad_bert_inputs(examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)


In [10]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                             shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

In [ ]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

In [ ]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

In [ ]:
len(vocab)